<a href="https://colab.research.google.com/github/Vengadore/Notebooks/blob/master/Training_DiabeticRetinopathy_Dataset_on_Efficientnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Diabetic Retinopathy Detection

Kaggle has a large competition of Diabetic Retinopathy detection which can be found here:
https://www.kaggle.com/c/diabetic-retinopathy-detection/

Their dataset consists in 35126 images labeled from 0 to 4 according to the degree of Retinopathy.
An analysis of the data is provided in this notebook.

In [1]:
!rm -rf sample_data
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp -R -v /content/drive/My\ Drive/PDR/ ./

'/content/drive/My Drive/PDR/' -> './PDR'
'/content/drive/My Drive/PDR/trainLabels.csv' -> './PDR/trainLabels.csv'
'/content/drive/My Drive/PDR/.ipynb_checkpoints' -> './PDR/.ipynb_checkpoints'
'/content/drive/My Drive/PDR/train' -> './PDR/train'


KeyboardInterrupt: ignored

### Install dependencies

In [1]:
from IPython.display import clear_output

!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!pip install efficientnet_pytorch
clear_output(wait=False)

### Load annotations

In [1]:
import pandas as pd
import os

data = pd.read_csv("/content/drive/My Drive/PDR/trainLabels.csv")
# Append path
data['image'] = data['image'].apply(lambda x : os.path.join("/content/drive/My Drive/PDR/train",x+".jpeg"))
data.head()

,image,level
0,/content/drive/My Drive/PDR/train/10_left.jpeg,0
1,/content/drive/My Drive/PDR/train/10_right.jpeg,0
2,/content/drive/My Drive/PDR/train/13_left.jpeg,0
3,/content/drive/My Drive/PDR/train/13_right.jpeg,0
4,/content/drive/My Drive/PDR/train/15_left.jpeg,1


In [2]:
data.groupby('level').count()

,image
level,
0,25810
1,2443
2,5292
3,873
4,708


In [3]:
## Sample dataframes to make them even
level0 = data[data['level']==0].sample(708,random_state=42)
level1 = data[data['level']==1].sample(708,random_state=42)
level2 = data[data['level']==2].sample(708,random_state=42)
level3 = data[data['level']==3].sample(708,random_state=42)
level4 = data[data['level']==4].sample(708,random_state=42)

In [4]:
new_data = pd.concat((level0,level1,level2,level3,level4),axis=0).reset_index()
new_data.groupby('level').count()

,index,image
level,,
0,708,708
1,708,708
2,708,708
3,708,708
4,708,708


In [5]:
data = new_data.copy()
data.head()

,index,image,level
0,13639,/content/drive/My Drive/PDR/train/17123_right....,0
1,10003,/content/drive/My Drive/PDR/train/12616_right....,0
2,5196,/content/drive/My Drive/PDR/train/6541_left.jpeg,0
3,11487,/content/drive/My Drive/PDR/train/14418_right....,0
4,31342,/content/drive/My Drive/PDR/train/39598_left.jpeg,0


#### Split data

In [6]:
from sklearn.model_selection import train_test_split

#Split data
X_train, X_test, y_train, y_test = train_test_split(data['image'], data['level'], test_size=0.22, random_state=42)

## Definition of the model

In [7]:
from efficientnet_pytorch import EfficientNet
import torch

model = EfficientNet.from_pretrained('efficientnet-b0')
## Change efficientnet final layer
model._fc = torch.nn.Linear(in_features=1280,out_features=5,bias = True)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


In [8]:
from torchvision.transforms import Resize,ToTensor,Compose,Normalize
from torchvision.transforms import RandomHorizontalFlip,RandomVerticalFlip,RandomRotation,ColorJitter
from PIL import Image

transforms = Compose([RandomHorizontalFlip(),RandomVerticalFlip(),RandomRotation(15),
                      ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5)]) # Transformations for the training images

composed = Compose([Resize((224,312)), # Resize to a fit size for efficientnet
                    ToTensor(),  # Convert into sensor
                    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # Normalize image

### Training parameters

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device);
torch.manual_seed(17)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)

In [19]:
try:
    model = torch.load('/content/drive/My Drive/PDR/checkpoint8.ph', map_location=device)
except:
    print("No Checkpoint loaded")

In [20]:
classes = {0:[0,0,0,0,1],
           1:[0,0,0,1,0],
           2:[0,0,1,0,0],
           3:[0,1,0,0,0],
           4:[1,0,0,0,0]}

In [23]:
from tqdm import tqdm
import random

epochs = 10
batch_size = 64

for epoch in range(epochs):
    
    indexes = [idx for idx in range(len(X_train))]
    pbar = tqdm( range(len(X_train)//batch_size),ncols = 100)
    running_loss = 0.0
    running_acc = 0.0
    t = 0

    for step in pbar:
        # Load data
        idx = random.sample(indexes,batch_size)
        X = X_train.iloc[idx]
        y = y_train.iloc[idx]

        # Remove indexes
        [indexes.remove(i) for i in idx]

        # Load images
        try:
            images = [Image.open(File) for File in X]
        except:
            continue
        # Load y_true
        y_true = torch.LongTensor([c for c in y]).to(device)
        
        # Convert images to tensor
        x_batch = torch.FloatTensor().to(device)
        for image in images:
            P = transforms(image)
            P = composed(P).unsqueeze(0).to(device)
            x_batch = torch.cat((x_batch,P))

        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = model(x_batch)
        loss = criterion(outputs, y_true)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        t += batch_size

        _, preds = torch.max(outputs, 1)
        running_acc += torch.sum(preds == y_true).cpu().detach().numpy()
        acc = torch.sum(preds == y_true).cpu().detach().numpy()/batch_size;
        pbar.set_description("Epoch: {} Accuracy: {:0.5f} Loss: {:0.5f} ".format(epoch+1,running_acc/t,loss.item()))
    #Validation
    val_acc = 0.0
    val_loss = 0.0
    t = 0
    for point in range(len(X_test)//batch_size):
        with torch.no_grad():

            X = X_test.iloc[point*batch_size:(point+1)*batch_size]
            y = y_test.iloc[point*batch_size:(point+1)*batch_size]


            # Load images
            try:
                images = [Image.open(File) for File in X]
            except:
                continue
            # Load y_true
            y_true = torch.LongTensor([c for c in y]).to(device)
            
            # Convert images to tensor
            x_batch = torch.FloatTensor().to(device)
            for image in images:
                P = composed(image).unsqueeze(0).to(device)
                x_batch = torch.cat((x_batch,P))

            
            outputs = model(x_batch)
            loss = criterion(outputs, y_true)
            val_loss += loss.item()
            t += batch_size
            _, preds = torch.max(outputs, 1)
            val_acc += torch.sum(preds == y_true).cpu().detach().numpy()
    print("\n Validation -- Accuracy: {:0.5f} Loss: {:0.5f} ".format(val_acc/t,loss.item()))
    try:
        torch.save(model,"/content/drive/My Drive/PDR/checkpoint{}.ph".format(epoch))
    except:
        continue

In [10]:
torch.save(model,"checkpoint.ph")

## Evaluation

In [21]:
from sklearn.metrics import confusion_matrix

In [24]:
predicted = []
true_values = []
for point in range(len(X_test)//batch_size):
    with torch.no_grad():

        X = X_test.iloc[point*batch_size:(point+1)*batch_size]
        y = y_test.iloc[point*batch_size:(point+1)*batch_size]

        true_values.append(y)
        # Load images
        try:
            images = [Image.open(File) for File in X]
        except:
            continue
        # Load y_true
        y_true = torch.LongTensor([c for c in y]).to(device)
        
        # Convert images to tensor
        x_batch = torch.FloatTensor().to(device)
        for image in images:
            P = composed(image).unsqueeze(0).to(device)
            x_batch = torch.cat((x_batch,P))

        
        outputs = model(x_batch)
        loss = criterion(outputs, y_true)
        
        _, preds = torch.max(outputs, 1)
        predicted.append(preds)

In [25]:
true_values = np.array(true_values).reshape(-1)
dummy = torch.FloatTensor()
for tensor in predicted:
    dummy = torch.cat((tensor,dummy))
predicted = dummy.view(-1)

ValueError: ignored

In [47]:
confusion_matrix(true_values, predicted)

array([[27, 42, 38, 42, 31],
       [24, 47, 25, 30, 25],
       [28, 39, 33, 23, 31],
       [22, 32, 30, 29, 29],
       [21, 34, 35, 27, 24]])